<a href="https://colab.research.google.com/github/Irek21/nn_models/blob/master/DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import math

In [0]:
input_size = 3072
hidden_size = 400
neck_size = 128
num_classes = 784
num_epochs = 10
batch_size = 100
iterations = 10
learning_rate = 0.001

In [3]:
# trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trans = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = dsets.CIFAR10(
    root='./data',
    train=True,
    transform=trans,
    download=True
)

test_dataset = dsets.CIFAR10(
    root='./data',
    train=False,
    transform=trans
)

Files already downloaded and verified


In [0]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [0]:
class ConvClassyfier(nn.Module): 
    def __init__(self): 
        super(ConvClassyfier, self).__init__() 
        self.layer1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=2, stride=2, padding=1), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2))
        # self.layer6 = nn.Sequential(nn.Conv2d(512, kernel_size=))

        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(512, 1024) 
        self.fc2 = nn.Linear(1024, 768)
        self.fc3 = nn.Linear(768, 10)
    
    def forward(self, x):
        out = x.reshape(x.size(0), -1)  
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out) 
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out)) 
        out = F.log_softmax(self.fc3(out)) 
        return out

In [25]:
inds = torch.where(batch[1] != 9)
images, labels = batch[0][inds], batch[1][inds]
labels

tuple

In [18]:
%%time
model = ConvClassyfier().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

total_step = len(train_loader)
loss_list = []
acc_list = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
   for i, batch in enumerate(train_loader):
     indices = torch.where(batch[1] != 9)
     images, labels = batch[0][indices], batch[1][indices]
     images = images.to(device)
     labels = labels.to(device)

     outputs = model(images)
     loss = criterion(outputs, labels)
     loss_list.append(loss.item())

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     if i % batch_size == 0:
         print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
             .format(epoch + 1, num_epochs, i, total_step, loss))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/10], Step [0/500], Loss: 2.3001182079315186
Epoch [1/10], Step [100/500], Loss: 1.9738048315048218
Epoch [1/10], Step [200/500], Loss: 1.6955684423446655
Epoch [1/10], Step [300/500], Loss: 1.6077343225479126
Epoch [1/10], Step [400/500], Loss: 1.265928030014038
Epoch [2/10], Step [0/500], Loss: 1.2663938999176025
Epoch [2/10], Step [100/500], Loss: 1.122253656387329
Epoch [2/10], Step [200/500], Loss: 1.1345077753067017
Epoch [2/10], Step [300/500], Loss: 1.1319414377212524
Epoch [2/10], Step [400/500], Loss: 1.0036706924438477
Epoch [3/10], Step [0/500], Loss: 0.7905498147010803
Epoch [3/10], Step [100/500], Loss: 0.8822279572486877
Epoch [3/10], Step [200/500], Loss: 0.9122844338417053
Epoch [3/10], Step [300/500], Loss: 0.7652236223220825
Epoch [3/10], Step [400/500], Loss: 0.8392135500907898
Epoch [4/10], Step [0/500], Loss: 0.8010050058364868
Epoch [4/10], Step [100/500], Loss: 0.7716930508613586
Epoch [4/10], Step [200/500], Loss: 0.6454837918281555
Epoch [4/10], Step [

In [0]:
def shuffle(tensor):
  indices = torch.randperm(tensor.shape[0])
  return tensor[indices]

In [0]:
def concat(tensor1, tensor2):
  new_tensor = torch.cat((tensor1, tensor2), dim=0)
  new_tensor = shuffle(new_tensor)
  return new_tensor.view(-1, 6, 32, 32)

In [0]:
def conseq_erase(tensor, oddness=0):
  indices = [2 * i + oddness for i in range(start, tensor.shape[0] // 2)] # если хочешь использовать, необходимо дофиксить правую границу
  return tensor[indices]

In [0]:
class ConvDE(nn.Module):
    def __init__(self):
        super(ConvDE, self).__init__()
        # Encoder
        self.layer1 = nn.Sequential(nn.Conv2d(6, 32, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout() 
        self.fc2 = nn.Linear(8 * 8 * 64 + input_size, neck_size)
        # Decoder
        self.fc3 = nn.Linear(neck_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, input_size)
    
    def encode(self, x, y):
        out = self.layer1(torch.cat((x, y), dim=1))
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out)
        out = F.relu(self.fc2(torch.cat((out.view(-1, 8 * 8 * 64), y.view(-1, input_size)), dim=1)))
        return out
    
    def decode(self, x):
        out = F.relu(self.fc3(x))
        out = self.fc4(out)
        return out
    
    def forward(self, x, y):
        out = self.encode(x, y)
        out = self.decode(out)
        return out

In [12]:
%%time
deltaencoder = ConvDE().cuda()
optimizer = torch.optim.Adam(deltaencoder.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

total_step = len(train_loader)
loss_list = []
acc_list = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
   for i, batch in enumerate(train_loader):
     indices = torch.where(batch[1] == 9) # вообще-то 1 так-то
     real_batch_size = len(indices)
     oddness = real_batch_size % 2
     images = batch[0][indices[:real_batch_size // 2]].to(device) # .view(-1, 6, 32, 32)
     images_prev = batch[0][indices[real_batch_size // 2: real_batch_size - oddness]].to(device)

     out_images = deltaencoder(images, images_prev)
     loss = criterion(out_images, images.view(-1, input_size))
     loss_list.append(loss.item())

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     if i % batch_size == 0:
         print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
             .format(epoch + 1, num_epochs, i, total_step, loss))

Epoch [1/10], Step [0/500], Loss: 0.2651723623275757
Epoch [1/10], Step [100/500], Loss: 0.05930454283952713
Epoch [1/10], Step [200/500], Loss: 0.042376257479190826
Epoch [1/10], Step [300/500], Loss: 0.03582959994673729
Epoch [1/10], Step [400/500], Loss: 0.033575572073459625
Epoch [2/10], Step [0/500], Loss: 0.027594538405537605
Epoch [2/10], Step [100/500], Loss: 0.025781365111470222
Epoch [2/10], Step [200/500], Loss: 0.026697998866438866
Epoch [2/10], Step [300/500], Loss: 0.023722229525446892
Epoch [2/10], Step [400/500], Loss: 0.022613627836108208
Epoch [3/10], Step [0/500], Loss: 0.022327905520796776
Epoch [3/10], Step [100/500], Loss: 0.02812112309038639
Epoch [3/10], Step [200/500], Loss: 0.022324975579977036
Epoch [3/10], Step [300/500], Loss: 0.01885760761797428
Epoch [3/10], Step [400/500], Loss: 0.018979400396347046
Epoch [4/10], Step [0/500], Loss: 0.020746586844325066
Epoch [4/10], Step [100/500], Loss: 0.018360694870352745
Epoch [4/10], Step [200/500], Loss: 0.0164703

In [8]:
%%time
deltaencoder = ConvDE().cuda()
optimizer = torch.optim.Adam(deltaencoder.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

total_step = len(train_loader)
loss_list = []
acc_list = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
   for i, batch in enumerate(train_loader):
     indices = torch.where(batch[1] == 1) # вообще-то 1 так-то
     real_batch_size = len(indices)
     oddness = real_batch_size % 2
     images = batch[0][indices[:real_batch_size // 2]].to(device) # .view(-1, 6, 32, 32)
     images_prev = batch[0][indices[real_batch_size // 2: real_batch_size - oddness]].to(device)

     out_images = deltaencoder(images, images_prev)
     loss = criterion(out_images, images.view(-1, input_size))
     loss_list.append(loss.item())

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     if i % batch_size == 0:
         print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
             .format(epoch + 1, num_epochs, i, total_step, loss))

Epoch [1/10], Step [0/500], Loss: 0.2569674253463745
Epoch [1/10], Step [100/500], Loss: 0.06958465278148651
Epoch [1/10], Step [200/500], Loss: 0.0507856048643589
Epoch [1/10], Step [300/500], Loss: 0.04329947009682655
Epoch [1/10], Step [400/500], Loss: 0.04145628213882446
Epoch [2/10], Step [0/500], Loss: 0.04017101228237152
Epoch [2/10], Step [100/500], Loss: 0.04276104271411896
Epoch [2/10], Step [200/500], Loss: 0.03700626268982887
Epoch [2/10], Step [300/500], Loss: 0.04134209826588631
Epoch [2/10], Step [400/500], Loss: 0.03507407754659653
Epoch [3/10], Step [0/500], Loss: 0.04122506454586983
Epoch [3/10], Step [100/500], Loss: 0.03739850968122482
Epoch [3/10], Step [200/500], Loss: 0.04058579355478287
Epoch [3/10], Step [300/500], Loss: 0.03640119731426239
Epoch [3/10], Step [400/500], Loss: 0.038493622094392776
Epoch [4/10], Step [0/500], Loss: 0.03582102060317993
Epoch [4/10], Step [100/500], Loss: 0.04350222647190094
Epoch [4/10], Step [200/500], Loss: 0.038233306258916855


In [28]:
few_shot = torch.tensor([], device=device)
for i, batch in enumerate(train_loader):
  indices = torch.where(batch[1] == 9)
  images = batch[0][indices].to(device)
  few_shot = torch.cat((few_shot, images))
few_shot = few_shot[torch.round(torch.rand(10) * 5000).long()]
few_shot.shape

RuntimeError: ignored

In [26]:
new_dset = torch.tensor([], device=device)
for i in range(10):
  for j, batch in enumerate(train_loader):
    indices = torch.where(batch[1] == 1)
    images = batch[0][indices].to(device)
    batch_shot = few_shot[i].expand(indices[0].shape[0], 3, 32, 32)
    samples = deltaencoder(batch_shot, images)
    new_dset = torch.cat((new_dset, samples.view(-1, 3, 32, 32)))
new_dset.shape

RuntimeError: ignored

In [0]:
new_dset_norm = torch.tensor([], device=device) 
for i in range(5000):
  sample = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)).__call__(new_dset[i])
  new_dset_norm = torch.cat((new_dset_norm, sample.view(1, 3, 32, 32)))

new_dset_norm = torch.tensor(new_dset_norm, requires_grad=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [12]:
new_dset_labels = torch.ones(5000, dtype=int, device=device) * 9
new_dset_labels

tensor([9, 9, 9,  ..., 9, 9, 9], device='cuda:0')

In [0]:
def batch_generator(X, y, batch_size=1):
    X_batch = torch.zeros(batch_size, X.shape[1], X.shape[2], X.shape[3])
    y_batch = torch.zeros(batch_size)
  sample = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)).__call__(new_dset[i])
  new_dset_norm = torch.cat((new_dset_norm, sample.view(1, 3, 32, 32)))

new_dset_norm = torch.tensor(new_dset_norm, requires_grad=True)
    batch_num = X.shape[0] // batch_size
    for i in range(batch_num):
        X_batch = X[i * batch_size:(i + 1) * batch_size, :, :, :]
        y_batch = y[i * batch_size:(i + 1) * batch_size]
        yield (X_batch, y_batch)

In [21]:
%%time
# model = ConvClassyfier().cuda()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

loss_list = []
acc_list = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
total_step = 100

for epoch in range(5):
   truck_batch_generator = batch_generator(new_dset_norm, new_dset_labels, batch_size=50)
   for i, batch in enumerate(truck_batch_generator):
     images = batch[0].to(device)
     labels = batch[1].to(device)

     outputs = model(images)
     loss = criterion(outputs, labels)
     loss_list.append(loss.item())

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     if i % 50 == 0:
         print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
             .format(epoch + 1, 5, i, total_step, loss))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/5], Step [0/100], Loss: 56.02851867675781
Epoch [1/5], Step [50/100], Loss: 0.0
Epoch [2/5], Step [0/100], Loss: 0.0
Epoch [2/5], Step [50/100], Loss: 0.0
Epoch [3/5], Step [0/100], Loss: 0.0
Epoch [3/5], Step [50/100], Loss: 0.0
Epoch [4/5], Step [0/100], Loss: 0.0
Epoch [4/5], Step [50/100], Loss: 0.0
Epoch [5/5], Step [0/100], Loss: 0.0
Epoch [5/5], Step [50/100], Loss: 0.0
CPU times: user 4.98 s, sys: 1.92 s, total: 6.9 s
Wall time: 6.99 s


In [22]:
total = 0
correct = 0

for i, batch in enumerate(test_loader):
  indices = torch.where(batch[1] == 9)
  images, labels = batch[0][indices], batch[1][indices]
  images = images.to(device)
  labels = labels.to(device)
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1)  # Выбор лучшего класса из выходных данных: класс с лучшим счетом
  total += labels.size(0)                    # Увеличиваем суммарный счет
  correct += (predicted == labels).sum()     # Увеличиваем корректный счет
     
print('Accuracy of the network on the truck test images: %d %%' % (100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the truck test images: 100 %


In [24]:
total = 0
correct = 0

for i, batch in enumerate(test_loader):
  # indices = torch.where(batch[1] != 9)
  images, labels = batch[0], batch[1]
  images = images.to(device)
  labels = labels.to(device)
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1)  # Выбор лучшего класса из выходных данных: класс с лучшим счетом
  total += labels.size(0)                    # Увеличиваем суммарный счет
  correct += (predicted == labels).sum()     # Увеличиваем корректный счет
     
print('Accuracy of the network on the truck test images: %d %%' % (100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the truck test images: 10 %
